In [1]:
import json
from pprint import pprint

In [2]:
f = open('./data/sentences.json')
data = json.load(f)

In [3]:
sentences = {}
for sentence_number in data:
    sentences[sentence_number] = {}
    s = data[sentence_number]
    # orc = s["subject"] + s["clausal noun"] + s["clausal verb"] + s["verb"] + s["preposition"] + s["object"]
    # orc_adv = s["subject"] + s["clausal noun"] + s["clausal verb"] + s["clausal adverb"] + s["verb"] + s["preposition"] + s["object"]
    # orc_clausal_verb = s["subject"] + s["clausal noun"] + s["clausal phrasal verb"] + s["verb"] + s["preposition"] + s["object"]
    # orc_clausal_verb_adv = s["subject"] + s["clausal noun"] + s["clausal phrasal verb"] + s["clausal adverb"] + s["verb"] + s["preposition"] + s["object"]
    # src = s["subject"] + s["clausal verb"] + s["clausal noun"] + s["verb"] + s["preposition"] + s["object"]
    # src_adv = s["subject"] + s["clausal verb"] + s["clausal noun"] + s["clausal adverb"] + s["verb"] + s["preposition"] + s["object"]

    orc = s["subject"] + s["clausal noun"] + s["clausal verb"] + s["verb phrase"] + "."
    orc_adv = s["subject"] + s["clausal noun"] + s["clausal verb"] + s["clausal adverb"] + s["verb phrase"] + "."
    orc_clausal_verb = s["subject"] + s["clausal noun"] + s["clausal phrasal verb"] + s["verb phrase"] + "."
    orc_clausal_verb_adv = s["subject"] + s["clausal noun"] + s["clausal phrasal verb"] + s["clausal adverb"] + s["verb phrase"] + "."
    src = s["subject"] + s["clausal verb"] + s["clausal noun"] + s["verb phrase"] + "."
    src_adv = s["subject"] + s["clausal verb"] + s["clausal noun"] + s["clausal adverb"] + s["verb phrase"] + "."

    sentences[sentence_number]['ORC'] = orc
    sentences[sentence_number]['ORC adv'] = orc_adv
    sentences[sentence_number]['ORC clausal verb'] = orc_clausal_verb
    sentences[sentence_number]['ORC clausal verb adverb'] = orc_clausal_verb_adv
    sentences[sentence_number]['SRC'] = src
    sentences[sentence_number]['SRC adverb'] = src_adv

pprint(sentences)

{'1': {'ORC': 'The bus driver that the kids followed wondered about the '
              'location of a hotel.',
       'ORC adv': 'The bus driver that the kids followed during the day '
                  'wondered about the location of a hotel.',
       'ORC clausal verb': 'The bus driver that the kids laughed at wondered '
                           'about the location of a hotel.',
       'ORC clausal verb adverb': 'The bus driver that the kids laughed at '
                                  'during the day wondered about the location '
                                  'of a hotel.',
       'SRC': 'The bus driver that followed the kids wondered about the '
              'location of a hotel.',
       'SRC adverb': 'The bus driver that followed the kids during the day '
                     'wondered about the location of a hotel.'},
 '10': {'ORC': 'The actor that the director watched forgot a critical line.',
        'ORC adv': 'The actor that the director watched on the set forgot a

In [4]:
# # Clone the repository
# !git clone https://github.com/facebookresearch/colorlessgreenRNNs

# # Navigate to the src directory, check its contents
# %cd colorlessgreenRNNs/src
# !ls

# # Get the pretrained model:
# !wget https://dl.fbaipublicfiles.com/colorless-green-rnns/best-models/English/hidden650_batch128_dropout0.2_lr20.0.pt

# !mkdir ../data/lm/English
# !wget -P ../data/lm/English/ https://dl.fbaipublicfiles.com/colorless-green-rnns/training-data/English/train.txt
# !wget -P ../data/lm/English/ https://dl.fbaipublicfiles.com/colorless-green-rnns/training-data/English/test.txt
# !wget -P ../data/lm/English/ https://dl.fbaipublicfiles.com/colorless-green-rnns/training-data/English/valid.txt
# !wget -P ../data/lm/English/ https://dl.fbaipublicfiles.com/colorless-green-rnns/training-data/English/vocab.txt

# %cd colorlessgreenRNNs/src/language_models


In [5]:
%cd colorlessgreenRNNs/src/language_models

/home/liza/Projects/school/701/colorlessgreenRNNs/src/language_models


In [6]:
import torch
import torch.nn as nn
import numpy as np

torch.manual_seed(50360)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(50360)

model_ = None
fn = "../hidden650_batch128_dropout0.2_lr20.0.pt"
with open(fn, "rb") as model_f:
    model_ = torch.load(fn)
    
# Makes sure your model is loaded onto the GPU (should return true)
next(model_.parameters()).is_cuda

# Finally - load the model.
from model import RNNModel

# Construct a new RNNModel using PyTorch 1.x implementations of NN modules
model = RNNModel("LSTM", 50001, 650, 650, 2, 0.2, False)
# Copy over the trained weights from the model loaded in
model.load_state_dict(model_.state_dict())

model = model.cuda()


/home/liza/anaconda3/envs/IESL/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'model.RNNModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/liza/anaconda3/envs/IESL/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/liza/anaconda3/envs/IESL/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by acces

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
## Import dictionary_corpus, part of the colorlessgreenRNNs repoistory that has some use useful functions
import dictionary_corpus

data_path = "../../data/lm/English"
dictionary = dictionary_corpus.Dictionary(data_path)

sentence = "the key to the cabinets"
tokenized_sentence = sentence.split()
input = [dictionary.word2idx[w]  if w in dictionary.word2idx
                                    else dictionary.word2idx["<unk>"]
                  for w in tokenized_sentence]

for w in tokenized_sentence:
    if w not in dictionary.word2idx:
        print(w, ' not in vocab!')

input = torch.tensor(input, dtype=torch.long)
input = input.cuda()

## Extract the hidden and output layers at each input token:
cur_sentence_output, cur_sentence_hidden = model(input.view(-1, 1), # (sequence_length, batch_size).
                       model.init_hidden(1)) # one input at a time, thus batch_size = 1

## Size of the output for the current sentence: Length of the sequence (in words), batch size (1), and length of the vocabulary (50001)
## This is the score of each word in the vocabulary (N = 50001) as the next token in the sequence, for each of the 5 input positions.  
cur_sentence_output.size()

## The output at the final token represents the predictions about the next word in the sequence. 
## We can pull out the next word predictions explicitly, since that's what we care about.
next_word_scores = cur_sentence_output[-1].view(-1)

## What's the most likely next word?
## Get the maximum value, and the associated id, with the max method:
predicted_word, predicted_word_id = next_word_scores.max(0)

predicted_word_id = predicted_word_id.item() # get an int out of a 1x1 tensor. Remove this line to see the difference!
print("Most likely next word ID:",predicted_word_id)
print("Most likely next word:", dictionary.idx2word[predicted_word_id])

## Retrieve the score of a particular word
is_idx = dictionary.word2idx["is"]
are_idx = dictionary.word2idx["are"]

print("is score: ", next_word_scores[is_idx].item())
print("are score: ", next_word_scores[are_idx].item())

## Scores are uninterpretable on their own; we might want to convert them 
## to a probability distribution using softmax
import torch.nn.functional as F

next_word_dist = F.softmax(next_word_scores, dim=0)

print("is prob: ", next_word_dist[is_idx].item())
print("are prob: ", next_word_dist[are_idx].item())
